In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

<h1>Segmentation</h1>

In [2]:
#src = cv.imread('/Users/kolsha/Pictures/water_coins.jpg')
src = cv.imread('/Users/kolsha/Pictures/i_m_pumpkin_rick__stencil_by_batnamz-dbrxwip.png')

src_gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
src_gray = cv.medianBlur(src_gray, 5)
src = cv.cvtColor(src, cv.COLOR_BGR2RGB)





@interact(dist=(0.0001, 1.0, 0.01))
def segmentation(dist):
    
    img = np.copy(src)
    ret, thresh = cv.threshold(src_gray,0,255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # noise removal
    kernel = np.ones((3,3),np.uint8)
    opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations = 10)
    # sure background area
    sure_bg = cv.dilate(opening,kernel,iterations=13)
    # Finding sure foreground area
    dist_transform = cv.distanceTransform(opening,cv.DIST_L2, 5)
    ret, sure_fg = cv.threshold(dist_transform, dist * dist_transform.max(), 255, 0)
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(sure_bg,sure_fg)

    # Marker labelling
    ret, markers = cv.connectedComponents(sure_fg)
    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1
    # Now, mark the region of unknown with zero
    markers[unknown == 255] = 0 #unknown == 255

    markers = cv.watershed(img, markers)
    #img[markers == -1] = [255,0,0]
    
    markers_gray = np.array(markers - markers.min()).astype(src_gray.dtype)

    
    _, contoursT, hierarchyT = cv.findContours(markers_gray, cv.RETR_CCOMP, cv.CHAIN_APPROX_TC89_KCOS)
    
    cv.drawContours(img, contoursT, -1, (0,255,0), 2, hierarchy=hierarchyT)
    
    titles = ['Src', 'Markers', 'Result']#['Original Image', 'Tpl', 'C Image', 'C Tpl']
    images = [src, markers_gray, img]#[src, tpl, dst_src, dst_tpl]
    fig = plt.gcf()
    fig.set_size_inches(20, 15.5)
    for i in range(len(images)):
        plt.subplot(2, 3, i+1),
        plt.imshow(images[i])
        plt.title(titles[i])
        plt.xticks([]),
        plt.yticks([])
    plt.show()
    return (dist)
    
    
    
    

interactive(children=(FloatSlider(value=0.4901, description='dist', max=1.0, min=0.0001, step=0.01), Output())…